In [1]:
import numpy as np
import sys

In [2]:
# This should do what the ZeroOut function in the ZeoliteCLassifierFour.m accomplishes



# This should do what ZeoliteCLassifierFour.m accomplishes
'''This is the function that is taking too many recursions. 
I belive that is because I transcribed this incorrectly from MATLAB'''
def zeoc(Adjacency, Source, LengthOfCycle, CurrentPath=[0], UsedNodes=[0],  iteration=0, CurrentVertex=0):    
    global CycleList
    if iteration == 0:
        CycleList= []
        CurrentVertex = Source
        UsedNodes = Source
        CurrentPath = [source]     
        iteration += 1
        zeoc(Adjacency, Source, LengthOfCycle, CurrentPath, [UsedNodes], iteration, CurrentVertex)
    else:
        if iteration == 1:
            UsedNodes[0] = []
        iteration += 1
        if len(CurrentPath) == LengthOfCycle and CurrentPath[-1] == Source:
            CycleList.append(CurrentPath.copy())
            Cycle = CurrentPath.copy()
            Adjacency = zerout(CurrentPath, Adjacency)
        elif len(CurrentPath) < LengthOfCycle:
            CurrentNeighbors = [x for x in range(len(Adjacency[CurrentVertex,:])) if Adjacency[CurrentVertex,x] != 0 ]
            for i in range(len(CurrentNeighbors)):
                Used = False
                CurrentVertex = CurrentNeighbors[i]
                for j in range(len(UsedNodes)):
                    if CurrentNeighbors[i] == UsedNodes[j]:
                        Used = True
                if not Used:
                    CurrentPath.append(CurrentVertex)
                    UsedNodes.append(CurrentVertex)
                    NodeIndex = len(UsedNodes)-1
                    Cycles, Adjacency = zeoc(Adjacency, Source, LengthOfCycle, CurrentPath, UsedNodes, iteration, CurrentVertex)
                    UsedNodes[NodeIndex] = []
                    del CurrentPath[-1]
                    #CurrentPath[-1] = []
            mmm = 0
        elif len(CurrentPath) == LengthOfCycle and CurrentPath[-1] != Source:
            UsedNodes[-1] = []
            CurrentPath[-1] = []
    Cycles = CycleList
    return Cycles, Adjacency

def zerout(CurrentPath, Adjacency):
    NewAdjacency = Adjacency
    temp = CurrentPath.copy()
    del temp[0]
    del temp[0]
    del temp[-1]
    del temp[-1]
    GraphSize = max(Adjacency.shape)
    Z = np.zeros([1, GraphSize])
    Z1 = np.zeros([GraphSize,1])
    for i in range(len(temp)):
        NewAdjacency[temp[i],:] = 0
        NewAdjacency[:,temp[i]] = 0
    return NewAdjacency
       

# Testing with chabazite

In [29]:
from ase.io import read, write
from ase import neighborlist
atoms = read('CHA.cif')
atoms = atoms.repeat([2,2,2])

In [30]:
cuttoff = neighborlist.natural_cutoffs(atoms)
neighborList = neighborlist.NeighborList(cutoffs=cuttoff, self_interaction=False, bothways=True)
neighborList.update(atoms)
matrix = neighborList.get_connectivity_matrix(sparse=False)

In [34]:
m = matrix.copy()
source = 19
Class = []
pr = [4,6,8]
for i in pr:
    ring,m = zeoc(m,source,i*2+1)
    if ring:
        for i in ring:
            temp = i
            Class.append(int((len(temp)-1)/2))
Class.sort(reverse=True)
print(Class)

[8, 8, 4]


# Testing with MFI
Testing with Oxygen 2 and 120. I'm not certain yet if the oxygens in MFI can associate with more than 3 rings, but it seems possible. 

Oxygen 2 should be 10,6,5.

Visual inspection suggests Oxygen 120 should be a 6,6,5, but maybe also in a 10-MR.

When using this tool on an oxygen in a 10-MR, the tool goes very slowly. 

In [58]:
atoms = read('MFI.cif')
atoms = atoms.repeat([1,1,2])

In [59]:
cuttoff = neighborlist.natural_cutoffs(atoms)
neighborList = neighborlist.NeighborList(cutoffs=cuttoff, self_interaction=False, bothways=True)
neighborList.update(atoms)
matrix = neighborList.get_connectivity_matrix(sparse=False)

In [60]:
m = matrix.copy()
source = 2
Class = []
pr = [4,5,6,10]
for i in pr:
    ring,m = zeoc(m,source,i*2+1)
    if ring:
        for i in ring:
            temp = i
            Class.append(int((len(temp)-1)/2))
    if len(Class) == 3:
        break
Class.sort(reverse=True)
print(Class)

[10, 6, 5]


In [65]:
m = matrix.copy()
source = 120
Class = []
pr = [4,5,6,10]
for i in pr:
    ring,m = zeoc(m,source,i*2+1)
    if ring:
        for i in ring:
            temp = i
            Class.append(int((len(temp)-1)/2))
    if len(Class) == 3:
        break
Class.sort(reverse=True)
print(Class)

[6, 6, 5]
